<a href="https://colab.research.google.com/github/LordNJS/fai-practical/blob/main/Best_first_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Best First Search

In [ ]:
# count how many times the rules are broken
def heuristic(sudoku):
  rule_broken = 0
  for i in sudoku:
    row = set(i)
    for a in row:
      rule_broken += i.count(a) - 1

  sudoku_cols = []
  for i in range(9):
    col = []
    for a in range(9):
      col.append(sudoku[a][i])
    sudoku_cols.append(col)

  for i in sudoku_cols:
    col = set(i)
    for a in col:
      rule_broken += i.count(a) - 1

  sudoku_squares = []
  squares = [[],[],[]]
  for i in range(9):
    for j in range(9):
      squares[j//3].append(sudoku[i][j])
    if i % 3 == 2:
      sudoku_squares += squares
      squares = [[],[],[]]

  for i in sudoku_squares:
    square = set(i)
    for a in square:
      rule_broken += i.count(a) - 1

  return rule_broken

SyntaxError: incomplete input (ipython-input-2-4081248956.py, line 10)